In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier # KerasClassifier is for hyper parameter tuning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping



In [5]:
data=pd.read_csv(r"D:\deepLearningNLP\ANNClassification\Churn_Modelling.csv")
data.head()
##Preprocess the data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [6]:
#Encode categorical variables
labelEncoderGender=LabelEncoder()
data['Gender']=labelEncoderGender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
##Onehot encode 'Geography' column
onehotEncoderGeo=OneHotEncoder(handle_unknown='ignore')
geoEncoded=onehotEncoderGeo.fit_transform(data[['Geography']]).toarray()
geoEncoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [8]:
geoEncodeddf=pd.DataFrame(geoEncoded,columns=onehotEncoderGeo.get_feature_names_out(['Geography']))
geoEncodeddf

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
#Combine onehot encoded values with original data
data=pd.concat([data.drop('Geography',axis=1),geoEncodeddf],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
X= data.drop('Exited',axis=1)
y=data['Exited']

In [11]:
#Split the data in training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
#Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [14]:
#Save the encoders and scaler for later use
import pickle
with open('labelEncoderGenderHyperParameter.pkl','wb') as file:
    pickle.dump(labelEncoderGender,file)
with open('onehotEncoderGeoHyperParameter.pkl','wb') as file:
    pickle.dump(onehotEncoderGeo,file)
with open('scalerHyperParameter.pkl','wb')as file:
    pickle.dump(scaler,file)

In [15]:
#Define a function to create the model and try different paramters in KeradClassifier
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],))) #this is for the first layer

    for _ in range(layers-1): #This will get executed if we have more than one layer
        model.add(Dense(neurons,activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [19]:
#Create a Keras Classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [17]:
# Define the grid search parameters
param_grid={
    'model__neurons':[16,32,64,128],
    'model__layers':[1,2],
    'epochs':[50,100]

}

In [21]:
#Perform grid search
from sklearn.model_selection import GridSearchCV

grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=1,cv=3,verbose=1)
grid_result=grid.fit(X_train,y_train)

#Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


d:\deepLearningNLP\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\deepLearningNLP\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\deepLearningNLP\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\deepLearningNLP\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\deepLearningNLP\venv\Lib\site-package

KeyboardInterrupt: 